# Primero vamos a traer los datos de los tuits del taller anterior

In [2]:
import pandas as pd
import tweepy

In [3]:
#leer las llaves
keys = pd.read_csv("C:/Users/AndresFelipe/Documents/Talleres NLP/Taller 10 twitter/mis_llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [4]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
### Para el taller anterior escogí los tuits referentes a Steph Curry
### ingrese las coordenadas de San Francisco - California - USA
tweets = tweepy.Cursor(api.search,
                       q=['Steph Curry -RT'],
                       lang='es',
                       geocode='37.7272,-123.032,5000km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(2000)

In [6]:
# Objeto generador
t = [tweet for tweet in tweets]
t[0]

Status(_api=<tweepy.api.API object at 0x000000666EEBCFA0>, _json={'created_at': 'Wed May 26 17:52:03 +0000 2021', 'id': 1397611502404870148, 'id_str': '1397611502404870148', 'full_text': 'Tiene que volver el Steph Curry TikToker, todos los necesitamos https://t.co/lFGynnb03j', 'truncated': False, 'display_text_range': [0, 63], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1397609891674705925, 'id_str': '1397609891674705925', 'indices': [64, 87], 'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1397609891674705925/pu/img/MF8NOzCEnQwGgxd2.jpg', 'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1397609891674705925/pu/img/MF8NOzCEnQwGgxd2.jpg', 'url': 'https://t.co/lFGynnb03j', 'display_url': 'pic.twitter.com/lFGynnb03j', 'expanded_url': 'https://twitter.com/LucianoLanci/status/1397611502404870148/video/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 540, 'h': 960, 'resize': 'fit

In [7]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)

In [8]:
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,"Tiene que volver el Steph Curry TikToker, todo...",2021-05-26 17:52:03,Bogotá,Bernal,1397611502404870148,https://twitter.com/i/web/stuatus/139761150240...
1,mira vos ahora cameron payne es el mismísimo s...,2021-05-26 04:16:27,Bogotá,"Los Angeles, CA",1397406249281540096,https://twitter.com/i/web/stuatus/139740624928...
2,Soy el único que piensa que el hijo de Drake s...,2021-05-26 00:04:24,Bogotá,"Caracas, Venezuela",1397342822001414150,https://twitter.com/i/web/stuatus/139734282200...
3,@loraxxd @batixtg @husslev Curry no tiene fmvp...,2021-05-25 23:29:17,Bogotá,Argentina,1397333984758149121,https://twitter.com/i/web/stuatus/139733398475...
4,@PaDiabelM Steph Curry deguisé 😅😅,2021-05-25 00:16:54,Bogotá,Boston,1396983576948789248,https://twitter.com/i/web/stuatus/139698357694...


In [9]:
data.shape

(150, 6)

In [10]:
import re

In [11]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

In [12]:
def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [13]:
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions
0,"Tiene que volver el Steph Curry TikToker, todo...",2021-05-26 17:52:03,Bogotá,Bernal,1397611502404870148,https://twitter.com/i/web/stuatus/139761150240...,"Tiene que volver el Steph Curry TikToker, todo...",[https://t.co/lFGynnb03j],[],[]
1,mira vos ahora cameron payne es el mismísimo s...,2021-05-26 04:16:27,Bogotá,"Los Angeles, CA",1397406249281540096,https://twitter.com/i/web/stuatus/139740624928...,mira vos ahora cameron payne es el mismísimo s...,[],[],[]
2,Soy el único que piensa que el hijo de Drake s...,2021-05-26 00:04:24,Bogotá,"Caracas, Venezuela",1397342822001414150,https://twitter.com/i/web/stuatus/139734282200...,Soy el único que piensa que el hijo de Drake s...,[https://t.co/ym659mowHP],"[#Drake, #StephenCurry, #BILLBOARD, #Billboard...",[]
3,@loraxxd @batixtg @husslev Curry no tiene fmvp...,2021-05-25 23:29:17,Bogotá,Argentina,1397333984758149121,https://twitter.com/i/web/stuatus/139733398475...,"Curry no tiene fmvp pq la NBA no quería, si...",[],[],"[@loraxxd, @batixtg, @husslev]"
4,@PaDiabelM Steph Curry deguisé 😅😅,2021-05-25 00:16:54,Bogotá,Boston,1396983576948789248,https://twitter.com/i/web/stuatus/139698357694...,Steph Curry deguisé 😅😅,[],[],[@PaDiabelM]
...,...,...,...,...,...,...,...,...,...,...
145,Steph Curry lanzó para 6/19 FG (38%) y 3/13 3P...,2021-05-19 23:10:00,Bogotá,La Habana,1395154802426920973,https://twitter.com/i/web/stuatus/139515480242...,Steph Curry lanzó para 6/19 FG (38%) y 3/13 3P...,[https://t.co/HCInKSG6C5],[#PLAYIN],[]
146,"La clave para los Lakers para mí, en principio...",2021-05-19 22:58:07,Bogotá,Dominican Republic,1395151812072775680,https://twitter.com/i/web/stuatus/139515181207...,"La clave para los Lakers para mí, en principio...",[],[],[]
147,#NBA : están de acuerdo? : Jordan Poole The ‘X...,2021-05-19 19:54:17,Bogotá,Republica dominicana,1395105550417989637,https://twitter.com/i/web/stuatus/139510555041...,: están de acuerdo? : Jordan Poole The ‘X’ Fa...,[https://t.co/ZDRVcKJ91q],[#NBA],[]
148,será clave cada vez que lo intenten atrapar co...,2021-05-19 17:25:51,Bogotá,"Los Angeles, CA",1395068196168761344,https://twitter.com/i/web/stuatus/139506819616...,será clave cada vez que lo intenten atrapar co...,[],[],[]


In [14]:
data.to_csv('tuits_taller_12.csv', index=False)

# Desde aca inicia el Taller No. 12 - Analisis de sentimiento

In [18]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [19]:
vader = SentimentIntensityAnalyzer()

In [20]:
import pandas as pd

In [21]:
tuits = pd.read_csv("C:/Users/AndresFelipe/Documents/Talleres NLP/Taller12/tuits_taller_12.csv")
tuits.head()

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions
0,mira vos ahora cameron payne es el mismísimo s...,2021-05-26 04:16:27,Bogotá,"Los Angeles, CA",1397406249281540096,https://twitter.com/i/web/stuatus/139740624928...,mira vos ahora cameron payne es el mismísimo s...,[],[],[]
1,Soy el único que piensa que el hijo de Drake s...,2021-05-26 00:04:24,Bogotá,"Caracas, Venezuela",1397342822001414150,https://twitter.com/i/web/stuatus/139734282200...,Soy el único que piensa que el hijo de Drake s...,['https://t.co/ym659mowHP'],"['#Drake', '#StephenCurry', '#BILLBOARD', '#Bi...",[]
2,@loraxxd @batixtg @husslev Curry no tiene fmvp...,2021-05-25 23:29:17,Bogotá,Argentina,1397333984758149121,https://twitter.com/i/web/stuatus/139733398475...,"Curry no tiene fmvp pq la NBA no quería, si...",[],[],"['@loraxxd', '@batixtg', '@husslev']"
3,@PaDiabelM Steph Curry deguisé 😅😅,2021-05-25 00:16:54,Bogotá,Boston,1396983576948789248,https://twitter.com/i/web/stuatus/139698357694...,Steph Curry deguisé 😅😅,[],[],['@PaDiabelM']
4,Bob Myers se reunió con Steph Curry durante un...,2021-05-24 18:46:00,Bogotá,"Oakland, CA",1396900303967039491,https://twitter.com/i/web/stuatus/139690030396...,Bob Myers se reunió con Steph Curry durante un...,['https://t.co/2sJCAbfC2x'],[],[]


In [26]:
pd.set_option('display.max_colwidth', None)

tuits['sentimiento_vader'] = tuits['tweets'].apply(lambda valor: vader.polarity_scores(valor)['compound'])
tuits['sentimiento_textblob'] = tuits['tweets'].apply(lambda valor: TextBlob(valor).sentiment.polarity)
tuits[["tweets", "sentimiento_vader", "sentimiento_textblob"]].head()

,tweets,sentimiento_vader,sentimiento_textblob
0,mira vos ahora cameron payne es el mismísimo steph curry,0.0000,0.0
1,Soy el único que piensa que el hijo de Drake se parece a Steph Curry? #Drake #StephenCurry #BILLBOARD #BillboardTNT https://t.co/ym659mowHP,0.0000,0.0
2,"@loraxxd @batixtg @husslev Curry no tiene fmvp pq la NBA no quería, si viste las finales del 2015 todos pensabamos q era de Steph. Yo me quedé en shock ajajaja",-0.7184,0.0
3,@PaDiabelM Steph Curry deguisé 😅😅,0.6124,0.0
4,"Bob Myers se reunió con Steph Curry durante una hora ayer, y se reunió con Draymond Green durante 90 minutos hace un momento. https://t.co/2sJCAbfC2x",0.0000,-0.2


## Tuits mas negativos

In [31]:
# Tuit más negativo para la libreria vaderSentiment
tuits[['tweets','sentimiento_vader']].sort_values('sentimiento_vader')

,tweets,sentimiento_vader
119,CURRY SIENDO CURRY 🔥🔥\n\nAsí finalizó la primera mitad en el partido entre Lakers y Golden State en el Play In 😱😱 los Warriors dominan 55-42 a Los Ángeles \n\nSteph lidera con 15 puntos 🏀🔥 https://t.co/oSD3rkiYXO,-0.9371
59,@GSWarriorsLATAM y si no meta ni lo libres! pero viejo el juego de curry y de draymond esta siendo horrible cuantas perdidas a steph no le esta cayendo el tiro no le pitan una y de paso 5 perdidas y 3 asistencias si lo estan doble y triple marcando creo que por muy malo que es kent generan nada,-0.8553
19,No me encuentro mentalmente apto para ver estos playoffs sin Steph Curry ☹️ https://t.co/aGsrcNjtAG,-0.8020
107,"Con un triple a lo Steph Curry, LeBron James le dio la victoria a los Lakers sobre los Warriors y la clasificación a los playoffs de la #NBA. Una bestia 💥💥💥 https://t.co/0kUbQdKE4V",-0.7579
2,"@loraxxd @batixtg @husslev Curry no tiene fmvp pq la NBA no quería, si viste las finales del 2015 todos pensabamos q era de Steph. Yo me quedé en shock ajajaja",-0.7184
...,...,...
78,@SoldadoStranger Hoy es noche de Steph Curry ✌🏼,0.4939
3,@PaDiabelM Steph Curry deguisé 😅😅,0.6124
11,Nadie quiere decirlo pero yo lo diré: Damian Lillard mejor que Steph Curry… Punto !!! Bai 👋😅,0.7633
75,"Tengo mucha fe para el partido de hoy pero el cagazo está, no hay duda. Como siempre, la buena noticia es que para mi equipo juega Steph Curry y un grupo de muchachos que se la bancan siempre. \n\nDale Warriors! 💙💛",0.8172


In [34]:
# Tuit más negativo para la libreria textblob
tuits[['tweets','sentimiento_textblob']].sort_values('sentimiento_textblob')

,tweets,sentimiento_textblob
59,@GSWarriorsLATAM y si no meta ni lo libres! pero viejo el juego de curry y de draymond esta siendo horrible cuantas perdidas a steph no le esta cayendo el tiro no le pitan una y de paso 5 perdidas y 3 asistencias si lo estan doble y triple marcando creo que por muy malo que es kent generan nada,-1.000
83,"Está dando su opinión acerca de pq cree que Curry no es MVP, con argumentos totalmente válidos.\nHace poquísimo tiempo declaró que Steph era el segundo mejor base de la historia (!!!)\nPero bueno, vayamos a la fácil y peguemosle a Lillard. https://t.co/7DVBGLh81g",-1.000
31,"@okcargentina0 Si obvio, no son todos. Me llevo bien con un montón.\nDetesto cuando para ""inflar"" la imagen de Curry lo meten en alguna comparación con Lillard, Harden, Trae, étc, cuando Steph no necesita de ninguna de esas comparaciones para que sepan que es el mejor base de todos.",-0.800
70,Dillon Brooks anuló los primeros minutos a Curry.\n\nLo hizo presionando arriba ante las bloqueos para evitar el handoff y poniéndose siempre de cara a la línea de base para evitar los desmarques.\n\nTodo bien hasta que Steph hizo de las suyas y empezó a tomar ritmo.,-0.800
93,"Mae que increíble cómo queda demostrado que los Warriors cuando Curry está en descanso, son unos fucking troncos e inútiles, hoy literal fue Steph Curry vs Lakers😔🙄",-0.600
...,...,...
36,"Las rachas de Steph Curry no aparecieron en #DubNation. Ja Morant, se robó el show, lideró a Memphis y dejó sin #Playoffs a Golden State por 2do año. \n\nOtro partidazo de #NBAPlayIn, hasta el momento este nuevo formato es un éxito total 👏👏👏 https://t.co/sd9RmwMCVG",0.150
146,#NBA : están de acuerdo? : Jordan Poole The ‘X’ Factor In Golden State Warriors-Los Angeles Lakers Play-In CBS San Francisco https://t.co/ZDRVcKJ91q,0.300
119,CURRY SIENDO CURRY 🔥🔥\n\nAsí finalizó la primera mitad en el partido entre Lakers y Golden State en el Play In 😱😱 los Warriors dominan 55-42 a Los Ángeles \n\nSteph lidera con 15 puntos 🏀🔥 https://t.co/oSD3rkiYXO,0.300
30,Steph Curry es Messi y Golden State es Barcelona. Parecidos!,0.375


## Tuits mas positivos

In [35]:
# Tuit más positivo para la libreria vaderSentiment
tuits[['tweets','sentimiento_vader']].sort_values('sentimiento_vader').tail()

,tweets,sentimiento_vader
78,@SoldadoStranger Hoy es noche de Steph Curry ✌🏼,0.4939
3,@PaDiabelM Steph Curry deguisé 😅😅,0.6124
11,Nadie quiere decirlo pero yo lo diré: Damian Lillard mejor que Steph Curry… Punto !!! Bai 👋😅,0.7633
75,"Tengo mucha fe para el partido de hoy pero el cagazo está, no hay duda. Como siempre, la buena noticia es que para mi equipo juega Steph Curry y un grupo de muchachos que se la bancan siempre. \n\nDale Warriors! 💙💛",0.8172
69,Te amo Steph Curry😍😍😍😍😍,0.9325


In [36]:
# Tuit más positivo para la libreria textblob
tuits[['tweets','sentimiento_textblob']].sort_values('sentimiento_textblob').tail()

,tweets,sentimiento_textblob
36,"Las rachas de Steph Curry no aparecieron en #DubNation. Ja Morant, se robó el show, lideró a Memphis y dejó sin #Playoffs a Golden State por 2do año. \n\nOtro partidazo de #NBAPlayIn, hasta el momento este nuevo formato es un éxito total 👏👏👏 https://t.co/sd9RmwMCVG",0.150
146,#NBA : están de acuerdo? : Jordan Poole The ‘X’ Factor In Golden State Warriors-Los Angeles Lakers Play-In CBS San Francisco https://t.co/ZDRVcKJ91q,0.300
119,CURRY SIENDO CURRY 🔥🔥\n\nAsí finalizó la primera mitad en el partido entre Lakers y Golden State en el Play In 😱😱 los Warriors dominan 55-42 a Los Ángeles \n\nSteph lidera con 15 puntos 🏀🔥 https://t.co/oSD3rkiYXO,0.300
30,Steph Curry es Messi y Golden State es Barcelona. Parecidos!,0.375
80,"Jugadores con al menos tres temporadas en las que finalizaron en el Top 3 del MVP desde la 2010/11:\n\nLeBron James\nJames Harden\nKawhi Leonard\nKevin Durant\nSteph Curry\n\nDe todas esas campañas, LBJ sólo se bajó de los finalistas en tres (contando esta).",0.500


## Tuits positivos y negativos por cada libreria

In [42]:
# Tuits positivos y negativos para la libreria vaderSentiment
pos_count, neg_count = 0, 0


for num in tuits['sentimiento_vader']:

    if num >= 0:
        pos_count += 1
  
    else:
        neg_count += 1
          
print("Número de tuits positivos: ", pos_count)
print("Número de tuits negativos: ", neg_count)

Número de tuits positivos:  113
Número de tuits negativos:  38


In [43]:
# Tuits positivos y negativos para la libreria textblob
pos_count, neg_count = 0, 0


for num in tuits['sentimiento_textblob']:

    if num >= 0:
        pos_count += 1
  
    else:
        neg_count += 1
          
print("Número de tuits positivos: ", pos_count)
print("Número de tuits negativos: ", neg_count)

Número de tuits positivos:  139
Número de tuits negativos:  12


## Visualización

In [45]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
75,00:04:00,1
65,00:16:00,1
71,00:49:00,1
76,00:59:00,1
103,01:08:00,1
...,...,...
66,23:29:00,1
101,23:32:00,1
43,23:40:00,1
24,23:50:00,1


In [46]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [47]:
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [48]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)